# Artificial Intelligence - Fall 2022 - Homework 2

# DEADLINE 18th of December 2022

## _Resolution in First Order Predicate Logic_

## Introduction

The goal of this homework is to become familiar with the resolution's working  mechanism and its strategies.

**_Useful resources:_**

* AI Course #5

## First Step

Use the notebook from your previous laboratory (lab8) and click on **Download** as &rarr; **Python**, save the file as `Lab08.py`, in the same directory.
You should also include in your directory the file `LPTester.py`.

In [6]:
# TODO
from LPTester import *
from FOP_helpful_funcs import *

In [8]:
from copy import deepcopy
from functools import reduce

# In this cell there are several functions used internally

dummy = make_atom("P")
[and_name, or_name, neg_name] = [get_head(s) for s in [make_and(dummy, dummy), make_or(dummy, dummy), make_neg(dummy)]]
def pFail(message, f):
    print(message + " <" + str(f) + ">")
    return False
def check_term(T):
    if is_constant(T):
        return (get_value(T) is not None) or pFail("The value of the constant is None", T)
    if is_variable(T):
        return (get_name(T) is not None) or pFail("The name of the variable is None", T)
    if is_function_call(T):
        return not [t for t in get_args(T) if not check_term(t)] and \
            (get_head(T) is not None or pFail("Function is not callable", T))
    return pFail("Term is not one of constant, variable or function call", T)
def check_atom(A):
    if is_atom(A):
        return not [t for t in get_args(A) if not check_term(t)] and \
            (get_head(A) is not None or pFail("Predicate name is None", A))
    return pFail("Is not an atom", A)
def check_sentence(S):
    if is_atom(S):
        return check_atom(S)
    if is_sentence(S):
        if get_head(S) in [and_name, or_name]:
            return (len(get_args(S)) >= 2 or pFail("Sentence has too few operands", S)) \
                and not [s for s in get_args(S) if not check_sentence(s)]
        if get_head(S) == neg_name:
            return (len(get_args(S)) == 1 or pFail("Negative sentence has not just 1 operand", S)) \
                and check_sentence(get_args(S)[0])
    return pFail("Not sentence or unknown type", S)

def add_statement(kb, conclusion, *hypotheses):
    s = conclusion if not hypotheses else make_or(*([make_neg(s) for s in hypotheses] + [conclusion]))
    if check_sentence(s):
        kb.append(s)
        print("OK: Added statement " + print_formula(s, True))
        return True
    print("-- FAILED CHECK: Sentence does not check out <"+print_formula(s, True)+"><" + str(s) + ">")
    return False

var_no = 0;

def assign_next_var_name():
    global var_no
    var_no += 1
    return "v" + str(var_no)

def gather_vars(S):
    return [get_name(S)] if is_variable(S) else \
        [] if not has_args(S) else reduce(lambda res, a: res + gather_vars(a), get_args(S), [])

def make_unique_var_names(KB):
    global var_no
    var_no = 0
    return [substitute(S, {var: make_var(assign_next_var_name()) for var in gather_vars(S)}) for S in KB]           
            
def print_KB(KB):
    print("KB now:")
    for s in KB:
        print("\t\t\t" + print_formula(s, True))

In [9]:
# KB 1
# based on an example in Artificial Intelligence - A Modern Approach
KB_America = []
#0 Mr West is a US general
add_statement(KB_America, make_atom("USGeneral", make_const("West")))
#1 General Awesome is also a US general
add_statement(KB_America, make_atom("USGeneral", make_const("General_Awesome")))
#2 General Awesome is Awesome
add_statement(KB_America, make_atom("Awesome", make_const("General_Awesome")))
#3 Nono is an enemy of America
add_statement(KB_America, make_atom("Enemy", make_const("Nono"), make_const("America")))
#4 M1 is a type of missile
add_statement(KB_America, make_atom("Missile", make_const("M1")))
#5 Nono has the M1 missile
add_statement(KB_America, make_atom("Owns", make_const("Nono"), make_const("M1")))

#6 any US general is an American
add_statement(KB_America, make_atom("American", make_var("x")), make_atom("USGeneral", make_var("x")))
#7 any missle is a weapon
add_statement(KB_America, make_atom("Weapon", make_var("x")), make_atom("Missile", make_var("x")))
#8 if anyone owns a missile, it is General West that sold them that missile
add_statement(KB_America, make_atom("Sells", make_const("West"), make_var("y"), make_var("x")), make_atom("Owns", make_var("x"), make_var("y")), make_atom("Missile", make_var("y")))
#9 any American who sells weapons to a hostile is a criminal
add_statement(KB_America, make_atom("Criminal", make_var("x")), make_atom("Weapon", make_var("y")), make_atom("Sells", make_var("x"), make_var("y"), make_var("z")), make_atom("Hostile", make_var("z")), make_atom("American", make_var("x")))
#10 any enemy of America is called a hostile
add_statement(KB_America, make_atom("Hostile", make_var("x")), make_atom("Enemy", make_var("x"), make_const("America")))
#11 America is awesome if at least an American is awesome
add_statement(KB_America, make_atom("Awesome", make_const("America")), make_atom("American", make_var("x")), make_atom("Awesome", make_var("x")))

KB_America = make_unique_var_names(KB_America)

print_KB(KB_America)

OK: Added statement USGeneral(West)
OK: Added statement USGeneral(General_Awesome)
OK: Added statement Awesome(General_Awesome)
OK: Added statement Enemy(Nono, America)
OK: Added statement Missile(M1)
OK: Added statement Owns(Nono, M1)
OK: Added statement (6 5USGeneral(?x) American(?x))
OK: Added statement (6 5Missile(?x) Weapon(?x))
OK: Added statement (6 5Owns(?x, ?y) 5Missile(?y) Sells(West, ?y, ?x))
OK: Added statement (6 5Weapon(?y) 5Sells(?x, ?y, ?z) 5Hostile(?z) 5American(?x) Criminal(?x))
OK: Added statement (6 5Enemy(?x, America) Hostile(?x))
OK: Added statement (6 5American(?x) 5Awesome(?x) Awesome(America))
KB now:
			USGeneral(West)
			USGeneral(General_Awesome)
			Awesome(General_Awesome)
			Enemy(Nono, America)
			Missile(M1)
			Owns(Nono, M1)
			(6 5USGeneral(?v2) American(?v2))
			(6 5Missile(?v4) Weapon(?v4))
			(6 5Owns(?v9, ?v8) 5Missile(?v8) Sells(West, ?v8, ?v9))
			(6 5Weapon(?v12) 5Sells(?v16, ?v12, ?v14) 5Hostile(?v14) 5American(?v16) Criminal(?v16))
			(6 5Enem

In [10]:
# KB 2
KB_Faster = []

def the_greyhound():
    return make_const("Greg")

#0 horses are faster than dogs
add_statement(KB_Faster, make_atom("Faster", make_var("x"), make_var("y")), make_atom("Horse", make_var("x")), make_atom("Dog", make_var("y")))
#1 there is a greyhound that is faster than any rabbit
add_statement(KB_Faster, make_atom("Faster", make_function_call(the_greyhound), make_var("z")), make_atom("Rabbit", make_var("z")))
#2 Harry is a horse
add_statement(KB_Faster, make_atom("Horse", make_const("Harry")))
#3 Ralph is a rabbit
add_statement(KB_Faster, make_atom("Rabbit", make_const("Ralph")))
#4 Greg is a greyhound
add_statement(KB_Faster, make_atom("Greyhound", make_function_call(the_greyhound)))
#5 A greyhound is a dog
add_statement(KB_Faster, make_atom("Dog", make_var("y")), make_atom("Greyhound", make_var("y")))
#6 transitivity
add_statement(KB_Faster, make_atom("Faster", make_var("x"), make_var("z")),
              make_atom("Faster", make_var("x"), make_var("y")), make_atom("Faster", make_var("y"), make_var("z")))

KB_Faster = make_unique_var_names(KB_Faster)

print_KB(KB_Faster)

OK: Added statement (6 5Horse(?x) 5Dog(?y) Faster(?x, ?y))
OK: Added statement (6 5Rabbit(?z) Faster(<function the_greyhound at 0x0000016508637880>[], ?z))
OK: Added statement Horse(Harry)
OK: Added statement Rabbit(Ralph)
OK: Added statement Greyhound(<function the_greyhound at 0x0000016508637880>[])
OK: Added statement (6 5Greyhound(?y) Dog(?y))
OK: Added statement (6 5Faster(?x, ?y) 5Faster(?y, ?z) Faster(?x, ?z))
KB now:
			(6 5Horse(?v3) 5Dog(?v4) Faster(?v3, ?v4))
			(6 5Rabbit(?v6) Faster(<function the_greyhound at 0x0000016508637880>[], ?v6))
			Horse(Harry)
			Rabbit(Ralph)
			Greyhound(<function the_greyhound at 0x0000016508637880>[])
			(6 5Greyhound(?v8) Dog(?v8))
			(6 5Faster(?v13, ?v11) 5Faster(?v11, ?v14) Faster(?v13, ?v14))


In [11]:
KB_test = []
add_statement(KB_test, make_atom("Q", make_var("x")), make_atom("P", make_var("x")))
add_statement(KB_test, make_atom("P", make_const("A")))

KB_test = make_unique_var_names(KB_test)
print_KB(KB_test)

OK: Added statement (6 5P(?x) Q(?x))
OK: Added statement P(A)
KB now:
			(6 5P(?v2) Q(?v2))
			P(A)


### Task 1 - 3p

* Implement the `resolves` function, which receives two clauses (literal or disjunctions of literals) and returns `False` if the two clauses do not resolve, otherwise returns a tuple containing the solution literals from the two clauses and the substitution under which they resolve.

* Two clauses resolve if there is a positive literal in one clause and a negative literal in the other, and the atoms of the two literals unify.

In [15]:
def is_positive_literal(L):
    return is_atom(L)

def is_negative_literal(L):
    global neg_name
    return get_head(L) == neg_name and is_positive_literal(get_args(L)[0])

def is_literal(L):
    return is_positive_literal(L) or is_negative_literal(L)

def get_literals(f1,atoms=[]):
    # Recursive function to get a list with the literals of the clause
    
    if is_literal(f1):
        # If the f1 is a literal , added to the liste of literals and return it
        atoms.append(f1)
        return atoms
    else:
        # if the argument wasn´t a literal
        for arg in get_args(f1):
            # for each argument i the clause get the literals
            get_literals(arg,atoms)
        return atoms

def complementary(lit1,lit2):
    # Function to know if two clauses are complenteary

    # If they are the same , they aren´t complementary
    if lit1==lit2:
        return False
    else:
        # Get the atom of the negative literals
        if is_negative_literal(lit1):
            lit1=get_args(lit1)[0]
        if is_negative_literal(lit2):
            lit2=get_args(lit2)[0]
        # if the atoms are the same, they are complementary 
        if lit1==lit2:
            return True
        else:
            return False


def resolves(C1, C2):
    #print("testing " + print_formula(C1, True) + " and " + print_formula(C2, True))
    # Returns a tuple (literal-from-C1, literal-from-C2, substitution)
    # where literal-from-C1 and literal-from-C2 unify under substitution.

    # Create a list with the literasl of the clauses
    lits1=list(get_literals(C1,[]))
    lits2=list(get_literals(C2,[]))
    # For each combination of literals
    for lit1 in lits1:
        for lit2 in lits2:
            
            atom1=lit1
            atom2=lit2
            # Verify that the aren't complementay
            if complementary(lit1,lit2):
                # if they are complentary , return
                return (lit1,lit2,{})
            # Verify that no one is negative literal, is that case get the atom
            if not is_atom(atom1):
                atom1=get_args(lit1)[0]
            if not is_atom(atom2):
                atom2=get_args(lit2)[0]

            # Make the unificacion of the atoms
            unification=unify(atom1,atom2)
            # If the unification is possible , return
            if unification:
                resolution=(lit1,lit2,unification)
                return resolution 
    
    # if the unfication isn't possible, return False
    return False
        

# Test!
test_batch(4, globals())

>>> Test batch [4]
Test 0: OK, got < (USGeneral(General_Awesome), 5USGeneral(?v2), {?v2 -> General_Awesome} >
Test 1: OK, got < (Owns(Nono, M1), 5Owns(?v9, ?v8), {?v8 -> M1, ?v9 -> Nono} >
Test 2: OK, got < (5American(?v16), American(?v2), {?v16 -> ?v2} >
Test 3: OK, got < (Enemy(Nono, America), 5Enemy(?v18, America), {?v18 -> Nono} >
Test 4: OK, got < False >
Test 5: OK, got < False >
Test 6: OK, got < False >
Test 7: OK, got < False >
Test 8: OK, got < (Q(5), 5Q(5), {} >
Test 9: OK, got < False >
>>>  10 / 10 tests successful.


**Expected Output:**

```
>>> Test batch [4]
Test 0: OK, got < (USGeneral(General_Awesome), ~USGeneral(?v2), {?v2 -> General_Awesome} >
Test 1: OK, got < (Owns(Nono, M1), ~Owns(?v9, ?v8), {?v8 -> M1, ?v9 -> Nono} >
Test 2: OK, got < (~American(?v16), American(?v2), {?v2 -> ?v16} >
Test 3: OK, got < (Enemy(Nono, America), ~Enemy(?v18, America), {?v18 -> Nono} >
Test 4: OK, got < False >
Test 5: OK, got < False >
Test 6: OK, got < False >
Test 7: OK, got < False >
Test 8: OK, got < (Q(5), ~Q(5), {} >
Test 9: OK, got < False >
>>>  10 / 10 tests successful.
```

_Obs:_ To display the result of the resolution function between two clauses, we can use the `print_r` function.

In [16]:
# Prints a 5-tuple resolvent representation (see below)
def print_r(R):
    if R is None:
        print("no resolvent")
    else:
        print("resolvent: " + print_formula(R[2], True) + "/" + print_formula(R[3], True) \
              + " {" + ", ".join([(k + ": " + str(R[4][k])) for k in R[4]]) + "}" \
              + "\n\t\t in " + print_formula(R[0], True) + "\n\t\t and " + print_formula(R[1], True))

### Task 2 - 2p

* Implement the `new_clause` function, which returns a new clause based on a solver, represented as a tuple of 5 elements:
   * the 2 clauses that resolve;
   * the 2 literals, one from the first clause, and one from the second clause, which resolves (given as they are in the 2 clauses);
   * the substitution under which the two literals resolve.
   
* If the result of the resolution step is the empty clause, the function must return `VOID_CLAUSE`.

* Otherwise, the result must be a clause that:
   * contains all the literals in the clauses that resolved, but
   * does not contain literals that have resolved, and
   * has the substitution applied (the `substitute` function was already implemented in laboratory 8), and
   * does not contain the same literal multiple times.

In [17]:
VOID_CLAUSE = "The void clause"

def new_clause(resolvent):
    C1, C2, L1, L2, s = resolvent
    # Apply the substitution on each clause and literal
    c1=substitute(C1,s)
    c2=substitute(C2,s)
    l1=substitute(L1,s)
    l2=substitute(L2,s)

    # Extract the literasl from the substitued clauses to compare
    lit_c1=get_literals(c1,[])
    lit_c2=get_literals(c2,[])
    # Remove the literasl that resolves
    lit_c1.remove(l1)
    lit_c2.remove(l2)

    
    if lit_c1==[] and lit_c2==[]:
        # If the clauses are fully resolved , return VOID_CLAUSE
        return VOID_CLAUSE
    else:
        # If the clauses still have literals, combine them
        new_clause=lit_c1
        for lit2 in lit_c2:
            # For each literal in the C2
            if lit2 not in new_clause :
                
                # If its not repeated in C1, add it
                new_clause.append(lit2)
                
    # Getting a new clause from a list of literals

        if len(new_clause)>2:
            # If there are more than two literas make OR with *args
            new_clause=make_or(new_clause[0],new_clause[1],*new_clause[2:])
        elif len(new_clause)==2:
            # If there are exacty two literasl , make standar OR
            new_clause=make_or(new_clause[0],new_clause[1])
        else:
            # If there are just one literals , undo the list
            new_clause=new_clause[0]

        # Return a new_clause in a FORMULA form
        return new_clause
    # TODO

# Test!
test_batch(5, globals())

>>> Test batch [5]
Test 0: OK, got < The void clause >
Test 1: OK, got < Q(?y) >
Test 2: OK, got < 5Q(5) >
Test 3: OK
>>>  4 / 4 tests successful.


**Expected Output:**

```
>>> Test batch [5]
Test 0: OK, got < The void clause >
Test 1: OK, got < Q(?y) >
Test 2: OK, got < ~Q(5) >
Test 3: OK
>>>  4 / 4 tests successful.
```

### Task 3 - 5p

Implement the missing part of the `solve_problem` function, using a resolution strategy of your choice to determine two clauses that resolve, and adding the result of the resolution step to the knowledge base.

In [57]:
def print_stack(stack):
    print('stack of possibles sustitution not applied:')
    for _ in stack:
        print(_[0][4])

def solve_problem(hypotheses, conclusion):
    KB = hypotheses[:]
    KB = [make_neg(conclusion)] + KB # You can also add at the end (depending on strategy)
    Effort = 500 # You can increase the effort if necessary
    # Create a stack with possible changes to the current KB unapplied
    stack=[]

    while Effort > 0:
        Effort -= 1
        # Base case to the resolvent
        resolvent=None
        # To indicates taht we already have a change
        flag=False
        # List with the keys of the stack
        list_key=[]
        c1=[]
        c2=[]
        # Select two clauses Clause1 and Clause2 which have not been previously chosen.
        for clause1 in reversed(KB):
            for clause2 in reversed(KB):
                # For each combination of clauses that aren't the same
                if clause1!=clause2:
                    # Get a resolution
                    res=resolves(clause1,clause2)
                    # If there is a valid resolution
                    if res!=False and not flag:
                        # Make a copy of KB to no distrub future iterations
                        KB_cp=KB[:]
                        # Remove the clauses of the KB copied
                        KB_cp.remove(clause2)
                        KB_cp.remove(clause1)
                        # Build the resolvent
                        resolvent = (clause1,clause2,res[0],res[1],res[2])
                        # Indicate that we already have the sustitution
                        flag=True
                    # If we find a valid sustitution
                    if res!= False:
                        # ExExtract the sustitution
                        key_r=[k for k in resolvent[4].keys()]
                        key_res=[k for k in res[2].keys() ]
                        value_r=[v for v in resolvent[4].values()]
                        value_res=[v for v in res[2].values()]
                    # if we already have a sustitution , and another susutitution for the same variable is possible on the same KB
                    if res!=False and flag and key_r==key_res and value_r!=value_res and key_r not in list_key :
                        #Save the sustitution and the data base on the stack
                        list_key.append(key_r)
                        KB_copy=KB[:]
                        KB_copy.remove(clause1)
                        KB_copy.remove(clause2)
                        stack.append(((clause1,clause2,res[0],res[1],res[2]),KB_copy))
        #When finished the searching of resolution, update KB
        KB=KB_cp   
        if len(stack):
            print_stack(stack)
        if resolvent:
            print('Sustitution applied:',resolvent[4])
        # If the thre isn't a valid resolution and it isn't more posibilities in the stack, Failed
        if resolvent is None and len(stack)==0:
            print("Failed. No resolving clauses. Effort left " + str(Effort))
            return False
        # If the isn't a solution , but we have other alternative resolution untested
        elif resolvent is None and len(stack)>0:
            # Change the KB and the resolution from the stack and continue testing
            print("Failed. No resolving clauses. Effort left " + str(Effort))
            print('**************************************************')
            print('Change')
            resolvent,KB=stack.pop()
            print('Sustitution applied:',resolvent[4])
            
        # Calculate the new clause to be added and add it to the knowledge base
        C = new_clause(resolvent)
        #print(C)
        
        if C == VOID_CLAUSE:
            print("Done (effort left " + str(Effort) + ")")
            return True
        
        # update KB
        print("Added: " + print_formula(C, True))
        KB = [C] + KB

        print_KB(KB)
    print("Failed. Effort exhausted.")
                
        
print_KB(KB_test)
solve_problem(deepcopy(KB_test), make_atom("Q", make_const("A")))
print("==========================================")

print_KB(KB_America)
solve_problem(deepcopy(KB_America), make_atom("Criminal", make_const("West")))
print("==========================================")

print_KB(KB_Faster)
solve_problem(deepcopy(KB_Faster), make_atom("Faster", make_const("Harry"), make_const("Ralph")))
print("==========================================")

KB now:
			(6 5P(?v2) Q(?v2))
			P(A)
Sustitution applied: {'v2': [0, 'A']}
Added: Q(A)
KB now:
			Q(A)
			5Q(A)
Sustitution applied: {}
Done (effort left 498)
KB now:
			USGeneral(West)
			USGeneral(General_Awesome)
			Awesome(General_Awesome)
			Enemy(Nono, America)
			Missile(M1)
			Owns(Nono, M1)
			(6 5USGeneral(?v2) American(?v2))
			(6 5Missile(?v4) Weapon(?v4))
			(6 5Owns(?v9, ?v8) 5Missile(?v8) Sells(West, ?v8, ?v9))
			(6 5Weapon(?v12) 5Sells(?v16, ?v12, ?v14) 5Hostile(?v14) 5American(?v16) Criminal(?v16))
			(6 5Enemy(?v18, America) Hostile(?v18))
			(6 5American(?v20) 5Awesome(?v20) Awesome(America))
stack of possibles sustitution not applied:
{'v20': [1, 'v2', None]}
Sustitution applied: {'v20': [1, 'v16', None]}
Added: (6 5Awesome(?v16) Awesome(America) 5Weapon(?v12) 5Sells(?v16, ?v12, ?v14) 5Hostile(?v14) Criminal(?v16))
KB now:
			(6 5Awesome(?v16) Awesome(America) 5Weapon(?v12) 5Sells(?v16, ?v12, ?v14) 5Hostile(?v14) Criminal(?v16))
			5Criminal(West)
			USGeneral(Wes

_Note:_ The expected output is longer than this print, but its start should be similar to:

```
KB now:
			~Q(A)
			(V ~P(?v2) Q(?v2))
			P(A)
refreshing CList
resolvent: ~Q(A)/Q(?v2){v2: (0, 'A')}
		 in ~Q(A)
		 and (V ~P(?v2) Q(?v2))
Added: ~P(A)
KB now:
			~P(A)
			(V ~P(?v2) Q(?v2))
			P(A)
refreshing CList
resolvent: ~P(A)/P(A){}
		 in ~P(A)
		 and P(A)
Done (effort left 497)
```